# Project: NoSQL Data Modeling with Apache Cassandra
<img src="images/cassandralogo.png" width="250" height="250" align="middle">

#### Overview
The fictional company, Sparkify, wants be able to analyze their song play data for their streaming music app platform. Currently the song play data resides in a directory CSV files, so an ETL pipeline is needed before the data can be loaded into a database. The pipeline will scrape the CSV files from the event directory and combined them into a denormalized, flat CSV file. After the data is combined, it can be loaded into Apache Cassandra tables designed for the example queries provided by the firm's analytics team.

#### Table of Contents
- [Part 1 - ETL Pipeline for Pre-Processing the Files](#ETL-pipeline)
    + [Import Python Packages](#import-libraries)
    + [Creating list of filepaths to process original event csv data files](#scrape-filepaths)
    + [Processing the files to create the data file csv that will be used for Apache Casssandra tables](#combine-csv-files)
    + [Contents of CSV file](#check-csv)
- [Part II - Apache Cassandra queries](#cassandra-queries)
    + [Setup Cassandra environment](#cassandra-setup)
        - [Creating a Cluster](#creating-cluster)
        - [Create Keyspace](#create-keyspace)
        - [Set Keyspace](#set-keyspace)
        - [Set Cassandra's `row factory` to created `pandas_factory`](#set-row_factory)
    + [Queries](#queries)
        - [Query 1](#query1)
            + [CREATE TABLE for query](#query1-create)
            + [INSERT data from csv file](#query1-insert)
            + [SELECT statement for query](#query1-select)
        - [Query 2](#query2)
            + [CREATE TABLE for query](#query2-create)
            + [INSERT data from csv file](#query2-insert)
            + [SELECT statement for query](#query2-select)
        - [Query 3](#query3)
            + [CREATE TABLE for query](#query3-create)
            + [INSERT data from csv file](#query3-insert)
            + [SELECT statement for query](#query3-select)
    + [Drop the tables before closing out the sessions](#drop-tables)
    + [Close the session and cluster connection](#close-session)

<a id="ETL-pipeline"></a>
# Part I. ETL Pipeline for Pre-Processing the Files
The log files are currently located in the `/event_data` directory with naming in the form `YYYY-mm-dd-events.csv`. These files need to be combined into a single file for easier loading when it comes time to create tables.

<a id="import-libraries"></a>
#### Import Python packages

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

<a id="scrape-filepaths"></a>
#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


<a id="combine-csv-files"></a>
#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


<a id="check-csv"></a>
#### There should now be a CSV file titled `event_datafile_new.csv` located within the current working directory.
The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### The contents of `event_datafile_new.csv` can be checked by loading the data into a pandas `DataFrame`.

In [6]:
event_datafile_df = pd.read_csv('event_datafile_new.csv')

In [7]:
event_datafile_df.head(3)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Muse,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Supermassive Black Hole (Twilight Soundtrack V...,42
1,Beastie Boys,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Lighten Up,42
2,Shakira,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Pienso En Ti,42


<a id="cassandra-queries"></a>
# Part II. Apache Cassandra queries

Apache Cassandra was chosen for the database because it has high availability and partition tolerance. The price for these benefits is that there are no multi-table `JOIN`s like in a relational database, requiring specialized tables for each query. Therefore, to setup the database, more information about what specific queries is required. Due to lack of normalization in the database, the denormalized, flat CSV file created in the first step is perfect for creating the tables in Cassandra.

<a id="cassandra-setup"></a>
### Setup Cassandra environment

<a id="creating-cluster"></a>
#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

<a id="create-keyspace"></a>
#### Create Keyspace

In [9]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_events
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

<a id="set-keyspace"></a>
#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkify_events')
except Exception as e:
    print(e)

<a id="set-row_factory"></a>
#### Set Cassandra's `row factory` to created `pandas_factory`

In [11]:
def pandas_factory(columns, rows):
    return pd.DataFrame(rows, columns=columns)
session.row_factory = pandas_factory


## Create queries to ask the provided three questions of the data

### Note: Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Generate column numbers for csv `line[index]` statments

In [12]:
list(enumerate(event_datafile_df.columns))

[(0, 'artist'),
 (1, 'firstName'),
 (2, 'gender'),
 (3, 'itemInSession'),
 (4, 'lastName'),
 (5, 'length'),
 (6, 'level'),
 (7, 'location'),
 (8, 'sessionId'),
 (9, 'song'),
 (10, 'userId')]

<a id="query1"></a>
### Query 1: Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
This query is looking at what songs were played by session and when in the session, which looks like a song play history. Therefore, the table will have the name `song_play_history` and have a compound `PRIMARY KEY` of session ID and what item the song was during that session (item in session), since multiple songs are expected per session. The other information in the table is going to be what the Sparkify team is looking for -- artist, song title, and duration.

<a id="query1-create"></a>
#### `CREATE TABLE` for query

In [13]:
# Give the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS song_play_history "
query = query + "(session_id int, \
                  item_in_session int, \
                  artist text, \
                  song_title text, \
                  song_length decimal, \
                  PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

<a id="query1-insert"></a>
#### `INSERT` data from csv file

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_play_history (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

<a id="query1-select"></a>
#### `SELECT` statement for query

In [15]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length \
         FROM song_play_history \
         WHERE session_id = 338 \
         AND item_in_session = 4"
try:
    # get rows from query
    rows = session.execute(query)
except Exception as e:
    print(e)

rows._current_rows

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


<a id="query2"></a>
### Query 2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
This query is looking at song information of a user during a specific session, or user session song history. The session ID would be expected to be unique, but since `WHERE` requires the search term to be a partition key, the user ID needs to be part of the `PRIMARY KEY`. Therefore, the `user_session_song_history` table will need a compound `PRIMARY KEY` of `user_id` and `session_id`. Since the song plays need to be sorted by occurence within the session, the cluster column of `item_in_session` will be added.

<a id="query2-create"></a>
#### `CREATE TABLE` for query

In [16]:
# Query 2: Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session_song_history"
query = query + "(user_id int, \
                  session_id int, \
                  item_in_session int, \
                  artist text, \
                  song_title text, \
                  first_name text, \
                  last_name text, \
                  PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                        

<a id="query2-insert"></a>
#### `INSERT` data from csv file

In [17]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_song_history (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

<a id="query2-select"></a>
#### `SELECT` statement for query

In [18]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, item_in_session, first_name, last_name \
         FROM user_session_song_history \
         WHERE user_id = 10 \
         AND session_id = 182"
try:
    # get rows from query
    rows = session.execute(query)
except Exception as e:
    print(e)

rows._current_rows

,artist,song_title,item_in_session,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


<a id="query3"></a>
### Query 3: Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
This query checks which users played a specific song, so a good name would be `who_played_a_song`. In order to have a unique `PRIMARY KEY`, the unique `user_id` can be added to the `song_title` to create a unique compound key. Using the `user_id` instead of the first and last name columns will ensure all users, including those with the same name, are reported. The query doesn't require the names to be ordered, so no need to worry about clustering columns.

<a id="query3-create"></a>
#### `CREATE TABLE` for query

In [19]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS who_played_a_song"
query = query + "(song_title text, \
                  user_id int, \
                  last_name text, \
                  first_name text, \
                  PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                         

<a id="query3-insert"></a>
#### `INSERT` data from csv file

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO who_played_a_song (song_title, user_id, last_name, first_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[4], line[1]))

<a id="query3-select"></a>
#### `SELECT` statement for query

In [21]:
# SELECT statement to verify the data was entered into the table
query = "SELECT user_id, first_name, last_name \
         FROM who_played_a_song \
         WHERE song_title='All Hands Against His Own'"
try:
    # get rows from query
    rows = session.execute(query)
except Exception as e:
    print(e)

rows._current_rows

,user_id,first_name,last_name
0,29,Jacqueline,Lynch
1,80,Tegan,Levine
2,95,Sara,Johnson


<a id="drop-tables"></a>
### Drop the tables before closing out the sessions

In [22]:
tables_to_drop = ['song_play_history',
                  'user_session_song_history',
                  'who_played_a_song']
for table_name in tables_to_drop:
    query = "DROP TABLE IF EXISTS %s"
    try:
        session.execute(query % table_name)
    except Exception as e:
        print(e)

<a id="close-session"></a>
### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()